In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
accelerator

In [0]:
!pip install Pillow==4.0.0
!pip install PIL
!pip install image

In [0]:
from google.colab import drive
drive.mount('/gdrive')

In [0]:
!ls -la /gdrive/My\ Drive/ML

In [0]:
!cp /gdrive/My\ Drive/ML/dogscats_train.tar.gz .
!ls .
!tar -xvf dogscats_train.tar.gz

In [0]:
import torchvision.models as models
alexnet = models.alexnet()
alexnet

In [0]:
vgg = models.vgg16_bn(pretrained=True)
vgg

In [0]:
import torch.nn as nn

num_classes = 2
vgg.classifier[-1] = nn.Linear(in_features=4096, out_features=num_classes)
vgg

In [0]:
from  torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import numpy as np

IMAGE_FOLDER_ROOT = 'sample_data/dogscats/train'

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

dataset = ImageFolder(IMAGE_FOLDER_ROOT,transform=transforms.Compose([transforms.Resize(256),transforms.RandomCrop(224),transforms.ToTensor(),normalize]))

validation_split = 0.25
indices = list(range(len(dataset)))
split = int(np.floor(validation_split * len(dataset)))
np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]
 
train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
valid_sampler = torch.utils.data.SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=16, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=16,
                                                sampler=valid_sampler)
print(len(train_indices),len(val_indices))

In [0]:
import torch.optim
import matplotlib.pyplot as plt
import numpy as np

def accuracy(output,labels):
  predictions = torch.argmax(output,dim=1)
  correct = (predictions == labels).sum().cpu().numpy()
  return correct / len(labels)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
vgg.to(device)
vgg.eval()

accuracies = []
with torch.no_grad():
  for itr,x in enumerate(validation_loader):
    data = x[0]
    labels = x[1]

    data = data.to(device)

    y_pred = vgg.forward(data)
    acc = accuracy(y_pred,labels)
    accuracies.append(acc)
    print(device,itr,'/',len(validation_loader),acc)


print('accuracy {:.2f}'.format(np.mean(accuracies)))

In [0]:
from torch.optim import lr_scheduler

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(vgg.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
vgg.to(device)
vgg.train()


for epoch in range(1):
  for itr,x in enumerate(train_loader):
    data = x[0]
    labels = x[1]

    optimizer.zero_grad()

    data = data.to(device)
    labels = labels.to(device)

    y_pred = vgg.forward(data)
    loss = criterion(y_pred,labels)

    loss.backward()
    optimizer.step()

    print('Iteration {}, train accuracy {:.2f}, loss {:.4f}'.format(itr+epoch*len(train_loader),accuracy(y_pred,labels),loss))

In [0]:
vgg.eval()

accuracies = []
with torch.no_grad():
  for itr,x in enumerate(validation_loader):
    data = x[0]
    labels = x[1]

    optimizer.zero_grad()

    data = data.to(device)
    labels = labels.to(device)

    y_pred = vgg.forward(data)
    accuracies.append(accuracy(y_pred,labels))
    

print('accuracy {:.2f}'.format(np.mean(accuracies)))